In [1]:
# pip install dash

In [2]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import layers
import matplotlib.pyplot as plt
import re

import nltk
from nltk.corpus import stopwords
import string
from  nltk import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.collocations import *
# nltk.download('wordnet')
from nltk import word_tokenize, FreqDist

from spacy import displacy
import spacy
from tqdm import tqdm

tqdm.pandas()
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

import plotly.graph_objects as go
import plotly.express as px


Using TensorFlow backend.


In [3]:
#Importing Webscraped data from Reddit and twitter talking about TSLA stock
df = pd.read_csv('static_dataset.csv')

In [4]:
#Shape of the Dataset
print("Shape of data=>",df.shape)
df.head(5)

Shape of data=> (10193, 6)


,Unnamed: 0,post,date,spacy,sentiment,subjectivity
0,0,$BBBY DD,2021-06-02,$BBBY DD,0.0,0.0
1,1,Here comes the sun turururu $RUN $TSLA,2021-06-02,Here comes the sun turururu $RUN $TSLA,0.0,0.0
2,2,$Run $Tsla f*** the suits,2021-06-02,$Run $Tsla f*** the suits,0.0,0.0
3,3,B.A.R.S - The Ultimate Meme Portfolio 🚀🚀🚀,2021-06-01,B.A.R.S - The Ultimate Meme Portfolio 🚀🚀🚀,0.0,1.0
4,4,(TSLA) vs (F),2021-06-01,(TSLA) vs (F),0.0,0.0


In [5]:
df=df.drop(['Unnamed: 0'], axis=1)

In [6]:
#Checking for Nulls
df.isnull().sum()

post            0
date            0
spacy           0
sentiment       0
subjectivity    0
dtype: int64

In [7]:
# taking a look at 5 random posts
for index,text in enumerate(df['post'][35:40]):
  print('Posts %d:\n'%(index+1),text)

Posts 1:
 $SOS clean and green till Elon gets a ring. WE SHALL RISE
Posts 2:
 Is $SPCE the next $GME.
Posts 3:
 F IS SUPER UNDERVALUED
Posts 4:
 $GOGO is a Starlink play
Posts 5:
 WOW! Look what is happening with Ford (F)! It have reached 15 years high and not stoping, I think it will be one of those stocks that get tremendous momentum! Let's watch!


In [8]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
df['post']=df['post'].apply(lambda x:expand_contractions(x))

In [9]:
# taking a look at 5 posts after expanding contrations
for index,text in enumerate(df['post'][35:40]):
  print('Posts %d:\n'%(index+1),text)

Posts 1:
 $SOS clean and green till Elon gets a ring. WE SHALL RISE
Posts 2:
 Is $SPCE the next $GME.
Posts 3:
 F IS SUPER UNDERVALUED
Posts 4:
 $GOGO is a Starlink play
Posts 5:
 WOW! Look what is happening with Ford (F)! It have reached 15 years high and not stoping, I think it will be one of those stocks that get tremendous momentum! Let is watch!


In [10]:
#Converting the posts into lower case for the purposes of NLP
df['post']=df['post'].apply(lambda x: x.lower())

In [11]:
#Removing numbers and words containing numbers
df['post']=df['post'].apply(lambda x: re.sub('\w*\d\w*','', x))

In [12]:
#Removing punctuation
df['post']=df['post'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

In [13]:
#Removing extra spaces
df['post']=df['post'].apply(lambda x: re.sub(' +',' ',x))

In [14]:
# taking a look at 5 random posts
for index,text in enumerate(df['post'][0:5]):
  print('Posts %d:\n'%(index+1),text)

Posts 1:
 bbby dd
Posts 2:
 here comes the sun turururu run tsla
Posts 3:
 run tsla f the suits
Posts 4:
 bars the ultimate meme portfolio 🚀🚀🚀
Posts 5:
 tsla vs f


## NLP:

In [15]:
#Lemmatizing the tokens and removing stopwords
df['lemmatized']=df['post'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [16]:
df_grouped=df[['post','lemmatized']].groupby(by='post').agg(lambda x:' '.join(x))
df_grouped.head()

,lemmatized
post,
,
nio will be worthless in a few years,nio worthless year
a day after a day recently only been trading for months now,day day recently trade month
a pop long tsla,pop long tsla
and a dream,dream


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
#Created Document tearm matrix
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(df_grouped['lemmatized'])
df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_dtm.index=df_grouped.index
df_dtm.head(3)

,aa,aaa,aaaaaaaaand,aaaaaaand,aaand,aal,aapl,aapltsla,aaxn,ab,...,𝐄𝐋𝐄𝐂𝐓𝐑𝐄𝐊,𝐇𝐀𝐒,𝐈𝐍,𝐈𝐍𝐂𝐑𝐄𝐀𝐒𝐄𝐃,𝐌𝐎𝐃𝐄𝐋,𝐎𝐅,𝐏𝐑𝐈𝐂𝐄,𝐓𝐄𝐒𝐋𝐀,𝐓𝐇𝐄,𝐔𝐒
post,,,,,,,,,,,,,,,,,,,,,
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nio will be worthless in a few years,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a day after a day recently only been trading for months now,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
df_dtm.astype('float')

post,,nio will be worthless in a few years,a day after a day recently only been trading for months now,a pop long tsla,and a dream,and do not know shit about options but i like this sub so fuck it bought puts on tsla,and i are offering the hedge fund managers who lost billions on gamestop free pillows so they have something to cry into at night,and some gratitude,annual wallstreetbets awards winners,april market crap,...,🙏 the ❤️ of god take your profits on tsla,🚀🚀🚀 tsla arkk httpstco,🚀🚀🚀 wave to tsla on your way to the moon 🚀🚀🚀,🚀🚀🚀🚀🚀🚀🚀tsla🚀🚀🚀🚀🚀🚀🚀,🚨🚨they are attacking daddy elon for revenge on his tweets if u have spare 💎 buy gme and tsla🚨🚨,🤑all in tsla🤑,🤗 this made my day thanks for the shoutout and congrats on your tsla 🎉 so inspiring to see retail investors be at the forefront and ahead of the curve tesla was a grassroots success story can’t stop smiling elonmusk,🦍 turn tsla to more 🍌🍌🍌,🦠poll time🦠 monday march,🦠poll time🦠monday march
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaaaand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
𝐎𝐅,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝐏𝐑𝐈𝐂𝐄,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝐓𝐄𝐒𝐋𝐀,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝐓𝐇𝐄,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# Importing wordcloud for plotting word clouds and textwrap for wrapping longer text
from wordcloud import WordCloud
from textwrap import wrap

# Function for generating word clouds
def generate_wordcloud(data,title):
    wc = WordCloud(width=400, height=330, max_words=150,colormap="Dark2").generate_from_frequencies(data)
    plt.figure(figsize=(20, 10))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.title('\n'.join(wrap(title,60)),fontsize=13)
    plt.show()
   
  
# Transposing document term matrix
df_dtm=df_dtm.transpose()

# Plotting word cloud for each product
# for index,product in enumerate(df_dtm.columns):
for index,post in enumerate(df.iloc[df['sentiment'].sort_values(ascending=False)[:3].index]['post']):
    generate_wordcloud(df_dtm[product].sort_values(ascending=False), product)

KeyError: 'aaa'

In [17]:
df.head()

,post,date,spacy,sentiment,subjectivity,lemmatized
0,bbby dd,2021-06-02,$BBBY DD,0.0,0.0,bbby dd
1,here comes the sun turururu run tsla,2021-06-02,Here comes the sun turururu $RUN $TSLA,0.0,0.0,come sun turururu run tsla
2,run tsla f the suits,2021-06-02,$Run $Tsla f*** the suits,0.0,0.0,run tsla f suit
3,bars the ultimate meme portfolio 🚀🚀🚀,2021-06-01,B.A.R.S - The Ultimate Meme Portfolio 🚀🚀🚀,0.0,1.0,bar ultimate meme portfolio 🚀 🚀 🚀
4,tsla vs f,2021-06-01,(TSLA) vs (F),0.0,0.0,tsla vs f


In [18]:
#Showing posts with the higest sentiment
print("3 Post with Highest Sentiment:")
for index,post in enumerate(df.iloc[df['sentiment'].sort_values(ascending=False)[:3].index]['post']):
  print('Post {}:\n'.format(index+1),post)

3 Post with Highest Sentiment:
Post 1:
 what ev stock is the best bet nio or arvl
Post 2:
 ⚠️breaking⚠️ tesla china 🇨🇳 sold china made model y in feb an impressive growth vs jan tsla httpstco
Post 3:
 tsla earnings report best quarter ever


In [19]:
#Post with the lowest sentiment
print("3 Post with Lowest Sentiment:")
for index,post in enumerate(df.iloc[df['sentiment'].sort_values(ascending=True)[:3].index]['post']):
  print('Post {}:\n'.format(index+1),post)

3 Post with Lowest Sentiment:
Post 1:
 i have buying power on robinhood what terrible position can i take on tsla before earnings
Post 2:
 to all the miserable bastards that missed the tsla rocket here is what your gainz could have looked like
Post 3:
 ohh stock market you are a cruel mistress
